In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import csv
from tqdm import tqdm

In [2]:
# Create and filter dataset with awards
awards = pd.read_csv('./awards.csv')#, sep = ',', header = 0, names = ['eventId', 'eventName', 'awardName', 'year','occurence', 'winAnnouncementTime', 'categoryName', 'nomeneeNote', 'name'])
awards = awards.dropna(subset = ['year', 'isWinner', 'const'])
awards = awards.drop(columns = ['occurrence',
 'eventName',
 'awardName',
 'name',
 'eventId',
 'winAnnouncementTime',
 'categoryName',
 'nomeneeNote',
 'originalName',
 'songNames',
 'episodeNames',
 'characterNames',
 'isPrimary',
 'isSecondary',
 'isPerson',
 'isCompany',
 'notes'])
awards.rename(columns={'const':'id_movie'}, inplace=True)
awards = awards[awards['isTitle']==True]
awards = awards.drop(columns = ['isTitle'])
awards

C:\Users\gonxh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,year,isWinner,id_movie
0,1992,True,tt0101428
2,1992,True,tt0101765
4,1992,True,tt0195135
6,2007,True,tt1337718
8,2007,True,tt1029440
...,...,...,...
2158516,1997,True,tt4020008
2158518,1997,True,tt4020008
2158520,1997,True,tt0124097
2158522,1997,True,tt2904162


In [3]:
# Create and filter dataset with movies
movies = pd.read_csv("IMDb_movies.csv")
movies = movies.dropna(subset = ['country', 'genre'])
movies['genre'] = movies['genre'].apply(lambda x : x.split(', '))
movies['country'] = movies['country'].apply(lambda x : x.split(', '))
movies = movies.drop(columns= ['original_title', 
   'year', 
   'title',
   'date_published',
   'duration',
   'language', 
   'director', 
   'writer',
   'production_company', 
   'actors', 
   'description', 
   'avg_vote', 
   'votes',
   'budget', 
   'usa_gross_income', 
   'worlwide_gross_income', 
   'metascore',
   'reviews_from_users', 
   'reviews_from_critics'])
movies = movies.explode('genre')
movies = movies.explode('country')
movies.rename(columns={'imdb_title_id':'id_movie'}, inplace=True)
movies

,id_movie,genre,country
0,tt0000574,Biography,Australia
0,tt0000574,Crime,Australia
0,tt0000574,Drama,Australia
1,tt0001892,Drama,Germany
1,tt0001892,Drama,Denmark
...,...,...,...
81269,tt9905412,Drama,India
81270,tt9905462,Drama,India
81271,tt9911774,Drama,India
81272,tt9914286,Drama,Turkey


In [4]:
# Merge both datasets
tqdm.pandas()
result = pd.merge(movies, awards, on=['id_movie'], how='inner').progress_apply(lambda x: x)

# Group some of the genres to reduce the list
droped_count = result.replace({'Biography': 'Documentary', 'Film-Noir':'Crime', 'Mystery':'Crime', 'Horror':'Thriller', 'Musical':'Music', 'Game-Show':'Reality_TV'})

# Create dataframe with "Genre, Country and Year" and count the number of movies in there
#droped_count = genreGrouped.groupby(['genre', 'country', 'year', 'isWinner']).size().to_frame('count').reset_index().rename(columns={'count':'numMovies'})

C:\Users\gonxh\Anaconda3\lib\site-packages\tqdm\std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  8.77it/s]


In [5]:
# Replace names of coutries that are not in the list
countries = droped_count['country'].value_counts()[:7].index.tolist()
droped_count.loc[~droped_count["country"].isin(countries), "country"] = "Other"

#Replace name of genres that are not in the list
genres = droped_count['genre'].value_counts()[:7].index.tolist()
droped_count.loc[~droped_count["genre"].isin(genres), "genre"] = "Other"

droped_count

,id_movie,genre,country,year,isWinner
0,tt0002199,Other,USA,1998,True
1,tt0002199,Drama,USA,1998,True
2,tt0002844,Crime,France,2011,False
3,tt0002844,Drama,France,2011,False
4,tt0003037,Crime,France,2011,False
...,...,...,...,...,...
1278774,tt9899880,Drama,Other,2019,False
1278775,tt9900782,Action,India,2020,True
1278776,tt9900782,Action,India,2020,True
1278777,tt9900782,Thriller,India,2020,True


In [6]:
# Create dataframe with countries and genres only in the list
new = droped_count.copy()
new = new[new['country'].isin(countries)]
new = new[new['genre'].isin(genres)]
new = new.reset_index().drop(columns=['index'])
new

,id_movie,genre,country,year,isWinner
0,tt0002199,Drama,USA,1998,True
1,tt0002844,Crime,France,2011,False
2,tt0002844,Drama,France,2011,False
3,tt0003037,Crime,France,2011,False
4,tt0003037,Drama,France,2011,False
...,...,...,...,...,...
632231,tt9872556,Comedy,Italy,2019,False
632232,tt9900782,Action,India,2020,True
632233,tt9900782,Action,India,2020,True
632234,tt9900782,Thriller,India,2020,True


In [7]:
new['range'] = pd.cut(new["year"], np.arange(1910, 2021, 10), labels=False)
lab = np.arange(1910, 2021, 10)
new['range'] = lab[new['range']]
new

,id_movie,genre,country,year,isWinner,range
0,tt0002199,Drama,USA,1998,True,1990
1,tt0002844,Crime,France,2011,False,2010
2,tt0002844,Drama,France,2011,False,2010
3,tt0003037,Crime,France,2011,False,2010
4,tt0003037,Drama,France,2011,False,2010
...,...,...,...,...,...,...
632231,tt9872556,Comedy,Italy,2019,False,2010
632232,tt9900782,Action,India,2020,True,2010
632233,tt9900782,Action,India,2020,True,2010
632234,tt9900782,Thriller,India,2020,True,2010


In [8]:
new.loc[new["isWinner"]==False, "isWinner"] = "Nominee"
new.loc[new['isWinner']==True, 'isWinner'] = 'Winner'
new.rename(columns={'genre':'Genre', 'country':'Country', 'isWinner':'Winner or Nominee'}, inplace=True)

In [9]:
new

,id_movie,Genre,Country,year,Winner or Nominee,range
0,tt0002199,Drama,USA,1998,Winner,1990
1,tt0002844,Crime,France,2011,Nominee,2010
2,tt0002844,Drama,France,2011,Nominee,2010
3,tt0003037,Crime,France,2011,Nominee,2010
4,tt0003037,Drama,France,2011,Nominee,2010
...,...,...,...,...,...,...
632231,tt9872556,Comedy,Italy,2019,Nominee,2010
632232,tt9900782,Action,India,2020,Winner,2010
632233,tt9900782,Action,India,2020,Winner,2010
632234,tt9900782,Thriller,India,2020,Winner,2010


In [10]:
new.to_csv('all_parallel_sets.csv')